# Create tracks database

In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm.notebook import tqdm
import pandas as pd
import spotipy

In [ ]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

## Read CSV

In [ ]:
playlists = pd.read_csv('../../data/sp_playlists.csv', index_col=0)

## Iterate through playlists to get tracks

In [ ]:
# We will need playlist ID
playlists['id'] = playlists.index

# Conversion
playlists = playlists.to_dict('records')

# Iteration
tracks = []
for playlist in tqdm(playlists):
    q = sp.playlist_tracks(playlist['id'])
    items = q['items'].copy()
    for item in items:
        item.update({'playlist_id': playlist['id']})
        if pd.isna(item['track']['id']):
            print(True)
    tracks.extend(items)
    while q['next']:
        q = sp.next(q)
        items = q['items'].copy()
        for item in items:
            item.update({'playlist_id': playlist['id']})
            if pd.isna(item['track']['id']):
                print(True)
        tracks.extend(items)

## Treat database

In [ ]:
# Filter
df = pd.DataFrame(tracks, columns=[
    'added_at',
    'added_by',
    'is_local',
    'primary_color',
    'track',
#     'video_thumbnail',
    'playlist_id',

])

In [ ]:
# Parse dates
df.added_at = pd.to_datetime(df.added_at)

In [ ]:
# Expand added_by column
df['added_by'] = df.added_by.apply(pd.Series).id

In [ ]:
# Expand track column

df2 = df.track.apply(pd.Series).copy()
df2 = df2[[
    'album',
    'artists',
    'available_markets',
    'disc_number',
    'duration_ms',
#     'episode',
    'explicit',
#     'external_ids',
#     'external_urls',
#     'href',
    'id',
#     'is_local',
    'name',
    'popularity',
#     'preview_url',
#     'track',
    'track_number',
#     'type',
#     'uri',
#     'linked_from'
]]
df = df.join(df2)
df.drop(columns='track', inplace=True)

In [ ]:
# Expand album column

df2 = df.album.apply(pd.Series).copy()
df2 = df2[[
    'album_type',
    'artists',
    'available_markets',
#     'external_urls',
#     'href',
    'id',
#     'images',
    'name',
    'release_date',
#     'release_date_precision',
#     'total_tracks',
#     'type',
#     'uri'
]]
df2.rename(columns={
    'artists': 'album_artists',
    'available_markets': 'album_available_markets',
    'id': 'album_id',
    'name': 'album_name',
    'release_date': 'album_release_date'
}, inplace=True)
df = df.join(df2)
df.drop(columns='album', inplace=True)

In [ ]:
# Expand artists column

df['artists_ids'] = df.artists.apply(lambda x: [i['id'] for i in x])
df['artists_names'] = df.artists.apply(lambda x: [i['name'] for i in x])
df.drop(columns='artists', inplace=True)

In [ ]:
# Expand album_artists column

df['album_artists_ids'] = df.album_artists.apply(lambda x: [i['id'] for i in x])
df['album_artists_names'] = df.album_artists.apply(lambda x: [i['name'] for i in x])
df.drop(columns='album_artists', inplace=True)

## Write CSV file

In [ ]:
df.to_csv('../../data/pd_tracks.csv')